# <h1>Задача классификации отзывов к кинофильмам с Kaggle. Подход Bag of Words</h1>

Ссылка на само соревнование: https://www.kaggle.com/c/word2vec-nlp-tutorial <br>
В нем есть подробный туториал, тут мы только резюмируем то, что там есть

Для начала загрузим все необходимые пакеты

In [1]:
import re
import logging
import time

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
    level=logging.INFO)

from bs4 import BeautifulSoup # удобная библиотека для обработки html-тегов, которые есть в текстах к этой задаче

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

from gensim.models import Word2Vec # библиотека gensim, в которой реализовано много Deep Learning алгоритмов
# в том числе есть много алгортмов для обработки текста, в том числе тематическое моделирование

import nltk
# nltk.download()  # важно скачать датасеты, в том числе стоп-слова
from nltk.corpus import stopwords # сразу забираем стоп-слова

# Часть 1: Пробуем bag of words

Считываем обучающую выборку

In [2]:
train = pd.read_csv("labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

Для начала надо убрать все числа из текстов, это можно сделать с помощью регулярных выражений, для этого есть библиотека **re**

In [4]:
letters_only = re.sub("[^a-zA-Z]",           # что искать
                      " ",                   # на что заменять
                      BeautifulSoup(train["review"][0]).get_text() )  # сам текст
print(letters_only)

 With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him The actual feature film bit when it finally starts is only on for    mi

Посмотрим на английские стоп-слова

In [5]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

Теперь напишем функцию, которая очищает текст

In [6]:
def review_to_words( raw_review ):
    # принимает на вход первоначальный текст, а возвращает строку из слов, разделенных пробелами
    #
    # 1. удаляем html-теги
    review_text = BeautifulSoup(raw_review).get_text() 
    #
    # 2. удаляем числа с помощью регулярных выражений        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. приводим слова к нижнему регистру и разбиваем текст на слова
    words = letters_only.lower().split()                             
    #
    # 4. понятно, что теперь мы по каждому из текстов должны пройтись и проверить его на совпадение с одним из стоп-слов.
    # т.е. по стоп-словам мы должны будем часто искать - для этого давайте положим стов слова в set (РАССКАЗАТЬ КАК РАБОТАЕТ SET)
    # и будем за O(log(n)) искать каждоый раз слово в set'е
    stops = set(stopwords.words("english"))                  
    # 
    # 5. удаляем стоп-слова
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. берем все слова, склеиваем в одну строку, добавляя пробелы
    return( " ".join( meaningful_words ))   

Теперь делаем тоже самое для всех текстов, которые у нас есть

In [7]:
num_reviews = train["review"].size
print("Cleaning and parsing the training set movie reviews...\n")
clean_train_reviews = []
# пробегаемся по всем текстам и запускаем для них review_to_words
for i in xrange( 0, num_reviews ):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Review %d of %d" % ( i+1, num_reviews ))                                                                 
    clean_train_reviews.append( review_to_words( train["review"][i] ))

Cleaning and parsing the training set movie reviews...

Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Review 25000 of 25000


Теперь будем строить Document Term Matrix. Тут все просто - мы берем CountVectorizer, на вход которого подаем максимально количество фич - они берутся после сортировки по частотности

С помощью fit_transform() мы во-первых строим словарь, во-вторых - обучающую выборку переделываем в Document Term Matrix

In [8]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000) 


train_data_features = vectorizer.fit_transform(clean_train_reviews)

# важно помнить, что sklearn на вход принимает numpy arrays, поэтому сконвертируем сразу
train_data_features = train_data_features.toarray()
print(train_data_features.shape)

(25000, 5000)


Можно отдельно посмотреть на словарь

In [11]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab[1:100])

[u'abc', u'abilities', u'ability', u'able', u'abraham', u'absence', u'absent', u'absolute', u'absolutely', u'absurd', u'abuse', u'abusive', u'abysmal', u'academy', u'accent', u'accents', u'accept', u'acceptable', u'accepted', u'access', u'accident', u'accidentally', u'accompanied', u'accomplished', u'according', u'account', u'accuracy', u'accurate', u'accused', u'achieve', u'achieved', u'achievement', u'acid', u'across', u'act', u'acted', u'acting', u'action', u'actions', u'activities', u'actor', u'actors', u'actress', u'actresses', u'acts', u'actual', u'actually', u'ad', u'adam', u'adams', u'adaptation', u'adaptations', u'adapted', u'add', u'added', u'adding', u'addition', u'adds', u'adequate', u'admire', u'admit', u'admittedly', u'adorable', u'adult', u'adults', u'advance', u'advanced', u'advantage', u'adventure', u'adventures', u'advertising', u'advice', u'advise', u'affair', u'affect', u'affected', u'afford', u'aforementioned', u'afraid', u'africa', u'african', u'afternoon', u'afte

Подсчитаем дополнительно частотности слов. Просто пробегаемся по словарю и смотрим, сколько раз очередное слово встречается в обучающей выборке

In [10]:
dist = np.sum(train_data_features, axis=0)

for count, tag in sorted([(count, tag) for tag, count in zip(vocab, dist)], reverse=True)[1:20]:
    print(count, tag)

40146 film
26788 one
20274 like
15140 good
12723 time
12646 even
12436 would
11983 story
11736 really
11474 see
10661 well
9765 much
9310 get
9301 bad
9285 people
9155 also
9061 first
9058 great
8362 made


Вполне логично, т.к. мы имеем дело с отзывами о фильмах

<h3>Тренируем randomForest</h3>

In [13]:
%%time
print("Training the random forest...")
# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100, n_obs=2j) 
# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few miфnutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

Training the random forest...
CPU times: user 4min 29s, sys: 32.8 s, total: 5min 1s
Wall time: 2min 35s


<h3>Теперь берем тестовую выборку и скорим ее</h3>

In [18]:
# читаем тестовые данные
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", \
                   quoting=3 )
# смотрим, что там реально 25 тыс слов и 2 колонки - id, text
print(test.shape)

num_reviews = len(test["review"])
clean_test_reviews = [] 

# дальше парсим каждый текст - запускаем review_to_words
print("Cleaning and parsing the test set movie reviews...")
for i in xrange(0,num_reviews):
    if( (i+1) % 5000 == 0 ):
        print("Review %d of %d" % (i+1, num_reviews))
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append( clean_review )

# т.к. словарь уже есть(ВАЖНО - СЛОВАРЬ ЗАНОВО СТРОИТЬ НЕНАДО, нужно делать НЕ fit_transform, а ПРОСТО transform!!!)
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

# передаем фичи натренированной модели
result = forest.predict(test_data_features)

# дальше создаем DataFrame с ответом и посылаем в систему
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv("Bag_of_Words_model.csv", index=False, quoting=3 )

(25000, 2)
Cleaning and parsing the test set movie reviews...
Review 5000 of 25000
Review 10000 of 25000
Review 15000 of 25000
Review 20000 of 25000
Review 25000 of 25000


В результате получаем в системе LB-score, равный 0.84384

In [61]:
print("Score 0.84384")

Score 0.84384


# Часть 2: используем word2vec для извлечения фичей

Основным отличием данного подхода является то, что нам не нужна обучающая выборка. Мы просто берем большое количество разных текстов и скармливаем их в word2vec структуру. Суть в том, что word2vec находит связи МЕЖДУ словами и тем самым для каждого слова находя векторное представление. 

**ОТЛИЧИЕ** заключается также в том, что в Bag Of Words мы считаем признаками сами слова и игнорируем связи, а в word2vec мы наиборот - игнорируем в целом сами слова (можно подавать на вход слова из разных тем и языков например), а признаками являются СВЯЗИ между словами!

In [30]:
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

In [25]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') # потребуется в дальнейшем для разбивки текстов на предложения

In [26]:
def review_to_wordlist( review, remove_stopwords=False ):
    # функция конвертирует первоначальный review, возвращая список слов
    # при этом, возможно удалить стоп-слова. 
    #
    # 1. удаляем HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. удаляем числа
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. приводим слова к нижнему регистру (что тоже в целом не обязательно) и разбиваем на слова
    words = review_text.lower().split()
    #
    # 4. удаляем стоп-слова (тоже опционально)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. возвращаем список слов
    return(words)

<h3>Напишем также функцию преобразования review в предложения (т.к. на вход к word2vec подаются предлжения)</h3>

In [27]:
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # разбиваем review на предложения. Возвращаем список предложений. Каждое предложение - список слов
    #
    # 1. NLTK Tokenizer требуется для того, чтобы разбить текст на предложения. Разбиваем на предложения
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. идем по каждому предложению
    sentences = []
    for raw_sentence in raw_sentences:
        # если предолжение пустое - пропускаем его
        if len(raw_sentence) > 0:
            # иначе запускаем review_to_wordlist и добавляем в sentences
            sentences.append( review_to_wordlist( raw_sentence, \
              remove_stopwords ))
            
    # возвращаем предложения
    return sentences

Теперь пробегаемся по всем review из обучающей выборки и преобразовываем каждое из них в предолжения

In [0]:
%%time
sentences = []  # Initialize an empty list of sentences
print("Parsing sentences from training set")
for i, review in enumerate(train["review"]):
    sentences += review_to_sentences(review, tokenizer)

Тоже самое делаем и с unlabeled_train - чем больше текст мы подаем на вход word2vec, тем лучше 

In [31]:
%%time
print("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

/usr/local/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/usr/local/lib/python2.7/site-packages/bs4/__init__.py:189: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.
  '"%s" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client to get the document behind the URL, and feed that document to Beautiful Soup.' % markup)
/usr/local/lib/

Parsing sentences from unlabeled set


In [33]:
# всего имеем вот столько предолжений
print(len(sentences))

857234


Посмотрим на некоторые предложения

In [37]:
print(sentences[0])
print(sentences[1])

[u'with', u'all', u'this', u'stuff', u'going', u'down', u'at', u'the', u'moment', u'with', u'mj', u'i', u've', u'started', u'listening', u'to', u'his', u'music', u'watching', u'the', u'odd', u'documentary', u'here', u'and', u'there', u'watched', u'the', u'wiz', u'and', u'watched', u'moonwalker', u'again']
[u'maybe', u'i', u'just', u'want', u'to', u'get', u'a', u'certain', u'insight', u'into', u'this', u'guy', u'who', u'i', u'thought', u'was', u'really', u'cool', u'in', u'the', u'eighties', u'just', u'to', u'maybe', u'make', u'up', u'my', u'mind', u'whether', u'he', u'is', u'guilty', u'or', u'innocent']


## Тренируем модель

В word2vec очень важно соблюсти баланс между всеми параметрами (которые определяют качество построенной модели) и временем обучения

* Architecture: обучение на skip-gramm'ах или на bag of words. Скип-граммы медленнее, но создают лучшее качество за счет того, что мы "шире" смотрим на окружение конкретного слова -> вектор признаков для него наиболее точный
* Training algorithm: сам алгоритм - Hierarchical softmax (по умолчанию) или negative sampling
* Downsampling of frequent words: убирать слова, которые встречаются очень не часто. Обычно по умолчанию 0.0001 работает хорошо
* Word vector dimensionality: Размерность получившегося пространства. Тут очень все критично к оперативной памяти - это во-первых, во-вторых, есть мнение (сам не тестировал), что больше 300 - не сильно влияет, с другой стороны - на некоторых форумах пишут, что вообще чем больше, тем лучше. Хз, чему верить больше. 
* Context / window size: сколько слов "вокруг" конкретного слова надо рассматривать. Обычно берут около 10 - в целом, норм
* Worker threads: равно обычно количеству ядер - 4-6
* Minimum word count: если слово встречается меньше стольки то раз во всех предложениях - его убираем - не путать с частотностью слова!

In [38]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 6          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

Тренируем модель

In [41]:
%%time
# Initialize and train the model (this will take some time)
print("Training model...")
model = Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)


Training model...
CPU times: user 9min 33s, sys: 11.3 s, total: 9min 45s
Wall time: 3min 17s


In [0]:
# если мы не собираемся больше перетренировывать модель - лучше сохранить ее закэшировать
model.init_sims(replace=True)

In [0]:
# модель можно сохранить
model_name = "300features_40minwords_10context"
model.save(model_name)

## <h3>Теперь поиграемся с тем, что получилось, пока забыв немного о задаче</h3>

Легко можно решать задачу doesnt_match - как это делается с помощью векторов?

Рассказать, что в спарке только это и есть и можно легко дописывать свои функции

In [42]:
model.doesnt_match("man woman child kitchen".split())

u'kitchen'

In [43]:
>>> model.doesnt_match("france england germany berlin".split())

u'berlin'

In [44]:
>>> model.doesnt_match("paris berlin london austria".split())

u'berlin'

Можно также запускать функию similar() - как это делается с помощью векторов?

In [45]:
>>> model.most_similar("man")

[(u'woman', 0.605443000793457),
 (u'guy', 0.5038936138153076),
 (u'boy', 0.4807789623737335),
 (u'men', 0.4526759088039398),
 (u'person', 0.44328293204307556),
 (u'himself', 0.4372999668121338),
 (u'girl', 0.4281119704246521),
 (u'lady', 0.4079314172267914),
 (u'son', 0.3900514543056488),
 (u'doctor', 0.3774733543395996)]

In [46]:
>>> model.most_similar("queen")

[(u'latifah', 0.4776698350906372),
 (u'victoria', 0.4730447828769684),
 (u'princess', 0.4706200659275055),
 (u'bee', 0.4386157691478729),
 (u'king', 0.4193441867828369),
 (u'prince', 0.40288692712783813),
 (u'throne', 0.394176185131073),
 (u'maria', 0.3907955586910248),
 (u'marie', 0.3768988847732544),
 (u'norma', 0.3682730197906494)]

In [47]:
>>> model.most_similar("awful")

[(u'terrible', 0.6598098278045654),
 (u'horrible', 0.6372771263122559),
 (u'dreadful', 0.6301497220993042),
 (u'atrocious', 0.5828288793563843),
 (u'horrendous', 0.5604714751243591),
 (u'abysmal', 0.5496130585670471),
 (u'laughable', 0.5353649854660034),
 (u'lousy', 0.5195952653884888),
 (u'horrid', 0.5079030990600586),
 (u'amateurish', 0.5057709217071533)]

# Часть 3 : как взять фичи из word2vec для конкретного документа?

## Подход 1 : средний вектор. Давайте возьмем документ

In [49]:
def makeFeatureVec(words, model, num_features):
    # берем документ и считаем средний вектор по всем словам
    # paragraph
    #
    # берем вектор, инициализируем изначально нулями
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word - содержит имена слов в словаре, чтобы по нему искать, лучше опять же, для скорости - положить его в set 

    index2word_set = set(model.index2word)
    #
    # бежим по каждому слову в документе и если слово встречается в словаре - добавляем его в ответ 
    # (прибавляем к результирующему вектору)
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # теперь соответственно делим на количество слов всего
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # эта функция берет на вход набор документов и для каждого из них возаращаетя средний вектор - полчается на выходе 2D-массив
    # 
    # инициализируем счетчик
    counter = 0.
    # 
    # точно также заполним нулями вектора (для скорости)
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # идем по всем ревью
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%5000. == 0.:
           print("Review %d of %d" % (counter, len(reviews)))
       # 
       # для каждого ревью считаем средний вектор
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # увеличиваем счетчик
       counter = counter + 1.
    return reviewFeatureVecs

In [51]:
%%time
clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

CPU times: user 18.8 s, sys: 1.23 s, total: 20.1 s
Wall time: 20.2 s


In [52]:
%time trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

Review 0 of 25000
Review 5000 of 25000
Review 10000 of 25000
Review 15000 of 25000
Review 20000 of 25000
CPU times: user 1min 18s, sys: 582 ms, total: 1min 19s
Wall time: 1min 19s


**ВНИМАНИЕ** Видно, что работает это все быстро,потому что для работы с обьектами линейно алгебры уже давно придумано много эффективных алгоритмов, которые позволяют многие вещи делать "на лету"

Дальше делаем тоже самое для тестовой выборки:

In [53]:
%%time
print("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

Creating average feature vecs for test reviews
CPU times: user 20.7 s, sys: 1.8 s, total: 22.5 s
Wall time: 22.7 s


In [54]:
%time testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 25000
Review 5000 of 25000
Review 10000 of 25000
Review 15000 of 25000
Review 20000 of 25000
CPU times: user 1min 18s, sys: 560 ms, total: 1min 19s
Wall time: 1min 19s


<h3>Дальше т.к. для каждого документа у нас есть набор фичей и есть обучающая выборка - можем применить supervised подход</h3>

In [57]:
forest = RandomForestClassifier( n_estimators = 100, n_jobs=2)

print("Fitting a random forest to labeled training data...")
%time forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...
CPU times: user 1min 3s, sys: 766 ms, total: 1min 3s
Wall time: 32.4 s
CPU times: user 1.08 s, sys: 25.8 ms, total: 1.11 s
Wall time: 768 ms


Видим, что результат получился примерно такой же, как в bag of words

In [60]:
print("Score 0.83248")

Score 0.83248


## Подход 2 : кластеризация

Давайте сделаем кластеризацию словаря. Будем считать количество кластеров как размер словаря / 5, так, чтобы в каждом кластере было примерно около 5 слов

Дальше возьмем первые 10 кластеров (тут можно выбирать - каких имеенно)

Теперь фичами будут количество слов из документа, которые принадлежат конкретному кластеру

In [58]:
%%time

word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

# делаем обычный k-means
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

CPU times: user 33min 29s, sys: 1min, total: 34min 30s
Wall time: 17min 41s


Кластеризация длится дотсаточно долго - следствие размерности и большого количества кластеров

In [59]:
word_centroid_map = dict(zip( model.index2word, idx ))

<h3>Давайте теперь посмотрим на сами кластеры, может быть они о чем-то скажут?</h3>

In [64]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print("\nCluster %d" % cluster)
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print(words)


Cluster 0
[u'pacific']

Cluster 1
[u'mutilated', u'screwing', u'shaken', u'slashed', u'knocked', u'punched', u'choking', u'yelled', u'fixing', u'shaved', u'melted', u'chopped', u'dumped', u'whacked', u'miraculously', u'waking', u'crushed', u'choked', u'sliced', u'drowned', u'bumped', u'handing', u'tracked', u'strangled', u'hypnotized', u'stripped', u'cooked', u'hammered', u'contaminated', u'busted', u'slammed', u'drugged', u'blasted', u'gunned', u'burnt', u'hung', u'swallowed', u'hacked', u'confessed', u'cleared', u'coma', u'cleaned', u'interrupted', u'bashed', u'gutted']

Cluster 2
[u'hilariously', u'shockingly', u'dreadfully', u'downright', u'amazingly', u'laughably', u'ridiculously', u'unbelievably', u'awfully']

Cluster 3
[u'maintained', u'sustained']

Cluster 4
[u'plunges', u'slices', u'boiling', u'cloak', u'dives', u'rises', u'melts', u'erupts', u'tosses', u'sinks', u'entrance', u'recovers', u'switches', u'piling', u'staircase', u'explosive', u'lays', u'sweeps', u'pushes', u'han

<h3>Теперь считаем фичи для каждого документа - фактически это можно назвать Bag Of Centroids</h3>

In [65]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    #
    # количество кластеров задается как максимум из значений
    # in the word / centroid map
    num_centroids = max( word_centroid_map.values() ) + 1
    #
    # опять же - заполняем нулями для скорости
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    #
    # Пробегаемся по каждому слову из ревью. Если очередное слово есть в словаре,
    # смотрим, какому кластеру оно принадлежит и увеличиваем счечик 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # возвращаем наш вектор, в i-ом элементе которого будет количество слов документа, которые принадлежат i-му кластеру
    return bag_of_centroids

In [66]:
# Pre-allocate an array for the training set bags of centroids (for speed)
train_centroids = np.zeros( (train["review"].size, num_clusters), \
    dtype="float32" )

# считаем фичи для обучающей выборки
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

# тоже самое делаем для тестовой выборки
test_centroids = np.zeros(( test["review"].size, num_clusters), \
    dtype="float32" )

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids( review, \
        word_centroid_map )
    counter += 1

In [68]:
# обучаем классификатор
forest = RandomForestClassifier(n_estimators = 100)

# преобразование занимает несколько минут
print("Fitting a random forest to labeled training data...")
forest = forest.fit(train_centroids,train["sentiment"])
result = forest.predict(test_centroids)

# сохраняем в файл и посылаем в систему
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [1]:
print("score 0.84648")

score 0.84648


Видим, что результат улучшился. Что делать дальше?

- Много играться с параметрами word2vec
- В кластерном подходе грамотно отобрать кластеры
- В среднем подходе считать среднее не по всем словам, а по тем, которые имеют наибольшую метрику TF-IDF

В целом, если посмотреть на решения с форума по задаче - можно видеть, что word2vec тут выигрывает в любом случае